## Loading & Extracting Meaningful Variables

In [50]:
import pandas as pd 
import numpy as np 

pd.set_option('display.max_columns', None)

In [51]:
df = pd.read_csv('../data/raw/occurrence.txt', sep='\t')
display(df.head(3))
df.shape

C:\Users\SABIO\AppData\Local\Temp\ipykernel_34408\3377765415.py:1: DtypeWarning: Columns (29,30,35,43,46,49,53,60,75,77,83,102,103,112,113,114,115,116,117,118,132,133,137,138,147,170,177) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/raw/occurrence.txt', sep='\t')


,gbifID,accessRights,bibliographicCitation,language,license,modified,publisher,references,rightsHolder,type,institutionID,collectionID,datasetID,institutionCode,collectionCode,datasetName,ownerInstitutionCode,basisOfRecord,informationWithheld,dataGeneralizations,dynamicProperties,occurrenceID,catalogNumber,recordNumber,recordedBy,recordedByID,individualCount,organismQuantity,organismQuantityType,sex,lifeStage,reproductiveCondition,caste,behavior,vitality,establishmentMeans,degreeOfEstablishment,pathway,georeferenceVerificationStatus,occurrenceStatus,preparations,disposition,associatedOccurrences,associatedReferences,associatedSequences,associatedTaxa,otherCatalogNumbers,occurrenceRemarks,organismID,organismName,organismScope,associatedOrganisms,previousIdentifications,organismRemarks,materialEntityID,materialEntityRemarks,verbatimLabel,materialSampleID,eventID,parentEventID,eventType,fieldNumber,eventDate,eventTime,startDayOfYear,endDayOfYear,year,month,day,verbatimEventDate,habitat,samplingProtocol,sampleSizeValue,sampleSizeUnit,samplingEffort,fieldNotes,eventRemarks,locationID,higherGeographyID,higherGeography,continent,waterBody,islandGroup,island,countryCode,stateProvince,county,municipality,locality,verbatimLocality,verbatimElevation,verticalDatum,verbatimDepth,minimumDistanceAboveSurfaceInMeters,maximumDistanceAboveSurfaceInMeters,locationAccordingTo,locationRemarks,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,coordinatePrecision,pointRadiusSpatialFit,verbatimCoordinateSystem,verbatimSRS,footprintWKT,footprintSRS,footprintSpatialFit,georeferencedBy,georeferencedDate,georeferenceProtocol,georeferenceSources,georeferenceRemarks,geologicalContextID,earliestEonOrLowestEonothem,latestEonOrHighestEonothem,earliestEraOrLowestErathem,latestEraOrHighestErathem,earliestPeriodOrLowestSystem,latestPeriodOrHighestSystem,earliestEpochOrLowestSeries,latestEpochOrHighestSeries,earliestAgeOrLowestStage,latestAgeOrHighestStage,lowestBiostratigraphicZone,highestBiostratigraphicZone,lithostratigraphicTerms,group,formation,member,bed,identificationID,verbatimIdentification,identificationQualifier,typeStatus,identifiedBy,identifiedByID,dateIdentified,identificationReferences,identificationVerificationStatus,identificationRemarks,taxonID,scientificNameID,acceptedNameUsageID,parentNameUsageID,originalNameUsageID,nameAccordingToID,namePublishedInID,taxonConceptID,scientificName,acceptedNameUsage,parentNameUsage,originalNameUsage,nameAccordingTo,namePublishedIn,namePublishedInYear,higherClassification,kingdom,phylum,class,order,superfamily,family,subfamily,tribe,subtribe,genus,genericName,subgenus,infragenericEpithet,specificEpithet,infraspecificEpithet,cultivarEpithet,taxonRank,verbatimTaxonRank,vernacularName,nomenclaturalCode,taxonomicStatus,nomenclaturalStatus,taxonRemarks,datasetKey,publishingCountry,lastInterpreted,elevation,elevationAccuracy,depth,depthAccuracy,distanceFromCentroidInMeters,issue,mediaType,hasCoordinate,hasGeospatialIssues,taxonKey,acceptedTaxonKey,kingdomKey,phylumKey,classKey,orderKey,familyKey,genusKey,subgenusKey,speciesKey,species,acceptedScientificName,verbatimScientificName,typifiedName,protocol,lastParsed,lastCrawled,repatriated,relativeOrganismQuantity,projectId,isSequenced,gbifRegion,publishedByGbifRegion,level0Gid,level0Name,level1Gid,level1Name,level2Gid,level2Name,level3Gid,level3Name,iucnRedListCategory
0,931073241,NaN,NaN,en,CC0_1_0,NaN,NaN,NaN,University of Guelph,NaN,NaN,NaN,http://dataset.canadensys.net/oac-specimens,Biodiversity Institute of Ontario,OAC,Ontario Agriculture College Herbarium (OAC-BIO),NaN,PRESERVED_SPECIMEN,NaN,NaN,NaN,41403,53588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PRESENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

(7240, 223)

In [42]:
df['continent'].value_counts()

continent
NORTH_AMERICA    6287
ASIA               15
Name: count, dtype: int64

In [45]:
df['verbatimLocality'].value_counts()

verbatimLocality
Linn County, OR, USA                                                                                                 80
Wells Gray Park, Thompson-Nicola, BC, CA                                                                             42
Central Kootenay, BC, Canada                                                                                         37
Squamish-Lillooet, BC, Canada                                                                                        32
Fraser Valley, BC, Canada                                                                                            28
                                                                                                                     ..
Lewis and Clark National Forest - Rocky Mountain District, Augusta, MT, US                                            1
Squamish, Squamish, BC, CA                                                                                            1
Flower Ridge           

## Dataset Cleaning

In [52]:
# Create list of columns from the comment
columns_to_keep = ['reproductiveCondition', 'occurrenceStatus', 
                  'eventDate', 'eventTime', 'startDayOfYear', 'endDayOfYear', 'year',
                  'month', 'day', 'verbatimEventDate', 'habitat', 'higherGeography',
                  'continent', 'countryCode', 'stateProvince', 'county', 'municipality',
                  'locality', 'verbatimLocality', 'verbatimElevation',
                  'minimumDistanceAboveSurfaceInMeters', 'maximumDistanceAboveSurfaceInMeters',
                  'decimalLatitude', 'decimalLongitude', 'coordinateUncertaintyInMeters',
                  'georeferencedDate', 'dateIdentified', 'elevation', 'elevationAccuracy',
                  'hasCoordinate', 'hasGeospatialIssues']

# Drop rows where continent is ASIA
df = df[df['continent'] != 'ASIA']

# Create a copy with only the specified columns
df_detailed = df[columns_to_keep].copy()

# Split columns into coordinate and location groups
coordinate_columns = ['decimalLatitude', 'decimalLongitude']
location_columns = ['countryCode', 'stateProvince', 'county', 'municipality', 'locality']

# Keep rows that have either:
# 1. Both coordinates (lat & long)
# 2. At least one location field filled
has_coordinates = df_detailed[coordinate_columns].notna().all(axis=1)
has_location = df_detailed[location_columns].notna().any(axis=1)

# Apply the filter to keep rows with either coordinates or location info
df_pruned = df_detailed[has_coordinates | has_location].copy()

print(f"\nOriginal shape: {df_detailed.shape}")
print(f"Shape after filtering: {df_pruned.shape}")
print(f"Removed {df_detailed.shape[0] - df_pruned.shape[0]} rows")

# Save pruned dataset
df_pruned.to_csv('../data/processed/pruned.csv', index=False)

# Display first few rows to verify the data
print("\nPruned dataset preview:")
# Display removed rows
removed_rows = df_detailed[~(has_coordinates | has_location)]
print("\nRemoved rows preview:")
display(removed_rows.head())

print("\nPruned dataset preview:")
display(df_pruned.head())


Original shape: (7225, 31)
Shape after filtering: (7002, 31)
Removed 223 rows

Pruned dataset preview:

Removed rows preview:


,reproductiveCondition,occurrenceStatus,eventDate,eventTime,startDayOfYear,endDayOfYear,year,month,day,verbatimEventDate,habitat,higherGeography,continent,countryCode,stateProvince,county,municipality,locality,verbatimLocality,verbatimElevation,minimumDistanceAboveSurfaceInMeters,maximumDistanceAboveSurfaceInMeters,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,georeferencedDate,dateIdentified,elevation,elevationAccuracy,hasCoordinate,hasGeospatialIssues
0,NaN,PRESENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
1,NaN,PRESENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
19,NaN,PRESENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
20,NaN,PRESENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
21,NaN,PRESENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False



Pruned dataset preview:


,reproductiveCondition,occurrenceStatus,eventDate,eventTime,startDayOfYear,endDayOfYear,year,month,day,verbatimEventDate,habitat,higherGeography,continent,countryCode,stateProvince,county,municipality,locality,verbatimLocality,verbatimElevation,minimumDistanceAboveSurfaceInMeters,maximumDistanceAboveSurfaceInMeters,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,georeferencedDate,dateIdentified,elevation,elevationAccuracy,hasCoordinate,hasGeospatialIssues
2,NaN,PRESENT,1970-07-11,NaN,192.0,192.0,1970.0,7.0,11.0,NaN,"scree slope, facing east",NaN,NORTH_AMERICA,CA,Alberta,NaN,Waterton Lakes National Park,West of Lost Lake,NaN,7000 feet,NaN,NaN,49.133333,-114.15,NaN,NaN,NaN,2133.0,0.0,True,False
3,NaN,PRESENT,1968-08-04,NaN,217.0,217.0,1968.0,8.0,4.0,NaN,Trail side,NaN,NORTH_AMERICA,CA,Alberta,NaN,Waterton Lakes National Park,"near campsite, north-west of Cameron Lake",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
4,NaN,PRESENT,1970-07-11,NaN,192.0,192.0,1970.0,7.0,11.0,NaN,"scree slope, facing east",NaN,NORTH_AMERICA,CA,Alberta,NaN,Waterton Lakes National Park,West of Lost Lake,NaN,7000 feet,NaN,NaN,49.133333,-114.15,NaN,NaN,NaN,2133.0,0.0,True,False
5,NaN,PRESENT,1959-07-10,NaN,191.0,191.0,1959.0,7.0,10.0,NaN,Coniferous forest,NaN,NORTH_AMERICA,US,Wyoming,NaN,NaN,Teton Pass,NaN,8400 feet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2560.0,0.0,False,False
6,NaN,PRESENT,1970-07-11,NaN,192.0,192.0,1970.0,7.0,11.0,NaN,"scree slope, facing east",NaN,NORTH_AMERICA,CA,Alberta,NaN,Waterton Lakes National Park,West of Lost Lake,NaN,7000 feet,NaN,NaN,49.133333,-114.15,NaN,NaN,NaN,2133.0,0.0,True,False


## Lon & Lat Cleaning

In [8]:
df_pruned[df_pruned['hasCoordinate'] == False].shape

(949, 31)

In [9]:
import sys
sys.path.append('..')
from src.geocoding import process_dataset
import pandas as pd
import os

# Process the dataset
df_updated, stats = process_dataset(df_pruned)

# Save the updated dataframe
output_path = '../data/processed/updated_coordinates.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_updated.to_csv(output_path, index=False)
print(f"\nSaved updated coordinates to: {output_path}")

# Show remaining rows without coordinates
print("\nRemaining rows without coordinates:")
display(df_updated[df_updated['decimalLatitude'].isna() & df_updated['decimalLongitude'].isna()])

# Show rows where LLM was used
print("\nRows where LLM was used for geocoding:")
display(df_updated[df_updated['used_llm']])

Geocoding progress: 100%|██████████| 949/949 [38:53<00:00,  2.46s/it]


Geocoding Statistics:
Total rows processed: 7017
Rows with missing coordinates: 949
Successfully updated: 848
Failed updates: 101
Rows where LLM was used: 130
Success rate: 89.36%

Detailed logs saved to: c:\Users\SABIO\Documents\GitHub\Capstone-Microsoft\notebooks\..\logs\geocoding_20250618_105637.log

Saved updated coordinates to: ../data/processed/updated_coordinates.csv

Remaining rows without coordinates:


,reproductiveCondition,occurrenceStatus,eventDate,eventTime,startDayOfYear,endDayOfYear,year,month,day,verbatimEventDate,habitat,higherGeography,continent,countryCode,stateProvince,county,municipality,locality,verbatimLocality,verbatimElevation,minimumDistanceAboveSurfaceInMeters,maximumDistanceAboveSurfaceInMeters,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,georeferencedDate,dateIdentified,elevation,elevationAccuracy,hasCoordinate,hasGeospatialIssues,used_llm
30,NaN,PRESENT,1800-01-01/2008-04-25,1800-01-01/2008-04-25,1.0,116.0,NaN,NaN,NaN,not recorded,NaN,no higher geography recorded,NaN,NaN,NaN,NaN,NaN,No specific locality recorded.,No verbatim locality recorded,NaN,NaN,NaN,NaN,NaN,NaN,2012-07-08 00:00:00,2008-04-25T00:00:00,NaN,NaN,False,False,False
1836,NaN,PRESENT,1967-07-26,NaN,207.0,207.0,1967.0,7.0,26.0,NaN,NaN,NaN,NORTH_AMERICA,CA,British Columbia,NaN,NaN,Aeta Lake.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
1865,NaN,PRESENT,1938-06-20,NaN,171.0,171.0,1938.0,6.0,20.0,NaN,NaN,NaN,NORTH_AMERICA,CA,British Columbia,NaN,NaN,"Near Lake Bootahnie, Marble Mountains.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1524.0,0.0,False,False,False
1885,NaN,PRESENT,1939-06-17,NaN,168.0,168.0,1939.0,6.0,17.0,NaN,NaN,NaN,NORTH_AMERICA,CA,British Columbia,NaN,NaN,"Sumup Cabin, southwest of Mount Baldy, 9 miles...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1326.0,46.0,False,False,False
1921,NaN,PRESENT,1963-07-06,NaN,187.0,187.0,1963.0,7.0,6.0,NaN,NaN,NaN,NaN,US,Idaho,NaN,NaN,"Ant Basin, Idaho, East Lago.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2439.0,0.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6466,NaN,PRESENT,1892-07-06,NaN,188.0,188.0,1892.0,7.0,6.0,NaN,NaN,NaN,NaN,US,Idaho,NaN,NaN,Owyhee Mountains,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
6675,NaN,PRESENT,1892-06-20,NaN,172.0,172.0,1892.0,6.0,20.0,NaN,NaN,NaN,NaN,US,Washington,NaN,NaN,[no locality given on the label].,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
6921,NaN,PRESENT,1929-08,NaN,NaN,NaN,1929.0,8.0,NaN,NaN,NaN,NaN,NaN,US,Washington,NaN,NaN,"Stevens Pass Region, Cascade Mountains.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
6945,NaN,PRESENT,1963-07-06,NaN,187.0,187.0,1963.0,7.0,6.0,NaN,NaN,NaN,NaN,US,Idaho,NaN,NaN,"Ant Basin, Idaho, East Lago.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False



Rows where LLM was used for geocoding:


,reproductiveCondition,occurrenceStatus,eventDate,eventTime,startDayOfYear,endDayOfYear,year,month,day,verbatimEventDate,habitat,higherGeography,continent,countryCode,stateProvince,county,municipality,locality,verbatimLocality,verbatimElevation,minimumDistanceAboveSurfaceInMeters,maximumDistanceAboveSurfaceInMeters,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,georeferencedDate,dateIdentified,elevation,elevationAccuracy,hasCoordinate,hasGeospatialIssues,used_llm
3,NaN,PRESENT,1968-08-04,NaN,217.0,217.0,1968.0,8.0,4.0,NaN,Trail side,NaN,NORTH_AMERICA,CA,Alberta,NaN,Waterton Lakes National Park,"near campsite, north-west of Cameron Lake",NaN,NaN,NaN,NaN,44.553159,-78.761708,NaN,NaN,NaN,NaN,NaN,False,False,True
29,NaN,PRESENT,1976-07-16,1976-07-16,198.0,198.0,1976.0,7.0,16.0,16 July 1976,NaN,"United States, Washington",NaN,US,Washington,NaN,NaN,"Chelan Co., Mazama: Along rocky shore of Lake ...","Chelan Co., Mazama: Along rocky shore of Lake ...",NaN,NaN,NaN,39.050541,-122.777656,NaN,1800-01-01 00:00:00,1800-01-01T00:00:00,NaN,NaN,False,False,True
67,NaN,PRESENT,1925-07-25T00:00,NaN,206.0,206.0,1925.0,7.0,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Seeley Lake, west side slough, north end",NaN,NaN,NaN,NaN,47.155299,-113.448428,NaN,NaN,NaN,NaN,NaN,False,False,True
1833,NaN,PRESENT,1941-07-08,NaN,189.0,189.0,1941.0,7.0,8.0,NaN,NaN,NaN,NORTH_AMERICA,CA,British Columbia,NaN,NaN,West side of Mountain Revelstoke.,NaN,NaN,NaN,NaN,50.998045,-118.195671,NaN,NaN,2006-04-06T00:00:00,1616.0,0.0,False,False,True
1849,NaN,PRESENT,1925-08,NaN,NaN,NaN,1925.0,8.0,NaN,NaN,NaN,NaN,NaN,US,Washington,NaN,NaN,"Inter Fork of White River, Mount Rainier.",NaN,NaN,NaN,NaN,46.852203,-121.757516,NaN,NaN,1944-01-01T00:00:00,1524.0,0.0,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6972,NaN,PRESENT,1896-06-19,NaN,171.0,171.0,1896.0,6.0,19.0,NaN,NaN,NaN,NaN,US,Washington,NaN,NaN,"Cascade Mountains, Mount Rainier and adjacent ...",NaN,NaN,NaN,NaN,46.852203,-121.757516,NaN,NaN,NaN,NaN,NaN,False,False,True
7013,NaN,PRESENT,1923-07-02,NaN,183.0,183.0,1923.0,7.0,2.0,NaN,NaN,NaN,NaN,US,Montana,NaN,NaN,Missoula National Forest: Montour Creek.,NaN,NaN,NaN,NaN,40.348990,-77.314162,NaN,NaN,NaN,NaN,NaN,False,False,True
7152,NaN,PRESENT,1942-06-10,NaN,161.0,161.0,1942.0,6.0,10.0,NaN,NaN,NaN,NaN,US,Montana,NaN,NaN,"Glacier National Park, south slope of Quartz R...",NaN,NaN,NaN,NaN,48.617486,-113.760885,NaN,NaN,NaN,NaN,NaN,False,False,True
7191,NaN,PRESENT,1938-05-14,NaN,134.0,134.0,1938.0,5.0,14.0,NaN,NaN,NaN,NaN,US,Idaho,NaN,NaN,Near South shore of Priest Lake.,NaN,NaN,NaN,NaN,48.609535,-116.878512,NaN,NaN,NaN,NaN,NaN,False,False,True


In [ ]:
import sys
sys.path.append('..')
from src.geocoding import process_dataset
import pandas as pd
import os

# Load your original dataset (assuming it's df_pruned or similar)
# Replace 'df_pruned' with whatever your original dataframe variable name is
df_original = df_pruned.copy()  # or whatever your original dataframe is called

# Process ALL rows that don't have coordinates with the updated logic
df_enhanced, stats = process_dataset(df_original)

# Save the updated dataframe
output_path = '../data/processed/updated_coordinates_with_verbatim.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_enhanced.to_csv(output_path, index=False)
print(f"\nSaved updated coordinates to: {output_path}")

# Show remaining rows without coordinates
print("\nRemaining rows without coordinates:")
display(df_enhanced[df_enhanced['decimalLatitude'].isna() & df_enhanced['decimalLongitude'].isna()])

# Show rows where LLM was used
print("\nRows where LLM was used for geocoding:")
display(df_enhanced[df_enhanced['used_llm']])

# Show some examples of verbatimLocality usage
print("\nExamples of verbatimLocality data:")
verbatim_examples = df_enhanced[df_enhanced['verbatimLocality'].notna()][['locality', 'verbatimLocality']].head(10)
display(verbatim_examples)

# Show cases where locality is empty but verbatimLocality has data
print("\nCases where locality is empty but verbatimLocality has data:")
empty_locality_with_verbatim = df_enhanced[
    (df_enhanced['locality'].isna() | 
     df_enhanced['locality'].str.lower().isin(['no specific locality recorded', 'no locality recorded', 'nan', 'none', ''])) &
    df_enhanced['verbatimLocality'].notna() &
    (~df_enhanced['verbatimLocality'].str.lower().isin(['no verbatim locality recorded', 'no locality recorded', 'nan', 'none', '']))
][['locality', 'verbatimLocality']].head(10)
display(empty_locality_with_verbatim)

# Show statistics
print(f"\nFinal Statistics:")
print(f"Total rows processed: {stats['total_rows']}")
print(f"Rows with missing coordinates: {stats['missing_coords']}")
print(f"Successfully updated: {stats['successfully_updated']}")
print(f"Failed updates: {stats['failed_updates']}")
print(f"Rows where LLM was used: {stats['llm_used_count']}")
success_rate = (stats['successfully_updated'] / stats['missing_coords'] * 100)
print(f"Success rate: {success_rate:.2f}%")

Geocoding progress:   0%|          | 0/934 [00:00<?, ?it/s]

Geocoding progress:   6%|▋         | 59/934 [02:38<43:39,  2.99s/it]

## Remaining Manual Work

In [22]:
import pandas as pd 
pd.set_option('display.max_columns', None)
test = pd.read_csv('../data/processed/updated_coordinates_2.csv')

test[test['decimalLatitude'].isna() & test['decimalLongitude'].isna()]['locality'].value_counts()

locality
Spanish Basin, Madison Range                                                                                  3
[no locality given on the label].                                                                             2
Missoula National Forest: Montour Creek.                                                                      2
Owyhee Mountains                                                                                              2
Near Lake Bootahnie, Marble Mountains.                                                                        1
Aeta Lake.                                                                                                    1
No specific locality recorded.                                                                                1
Nez Perce Natl. For.                                                                                          1
Near summit of the Metolias Trail, Cascade Mts.                                                

In [21]:
# Define clearly useless locality values
useless_localities = [
    "[no locality given on the label].",
    "No specific locality recorded.",
    "[no additional data]",
    "no locality recorded",  # add more as needed
]

# Drop rows with these locality values
test = test[~test['locality'].isin(useless_localities)]

# Re-check missing lat/lon
missing_coords = test[test['decimalLatitude'].isna() & test['decimalLongitude'].isna()]

In [23]:
missing_coords['locality'].value_counts()

locality
Spanish Basin, Madison Range                                                                                  3
Owyhee Mountains                                                                                              2
Missoula National Forest: Montour Creek.                                                                      2
Aeta Lake.                                                                                                    1
Near Lake Bootahnie, Marble Mountains.                                                                        1
Cascade Mtns                                                                                                  1
Nez Perce Natl. For.                                                                                          1
4 mi. E of Stevens Pass on U.S. 2 by turnoff in Spruce-Hemlock forest & alpine.                               1
"Sphinx glacier foreland, Garbildi park Locality Glacial moraines"                             

In [24]:
manual_geocodes = {
    "Spanish Basin, Madison Range": (45.1652, -111.3014),
    "Missoula National Forest: Montour Creek.": (46.8724, -114.8820),
    "Owyhee Mountains": (42.6197, -116.7894),
    "Near Lake Bootahnie, Marble Mountains.": (41.6528, -123.7283),
    "Aeta Lake.": (49.7683, -125.0020),
    '"Sphinx glacier foreland, Garbildi park Locality Glacial moraines"': (49.9615, -123.0135),
    "Cascade Mtns": (47.5000, -121.8000),
    "Meagher Co: in lodgepole forest. Harley Park, ca. 6 mi. west of Neihart, Little Belt Mts.": (46.9581, -110.8262),
    "Near summit of the Metolias Trail, Cascade Mts.": (44.4608, -121.8053),
    "Forest and Range Expt. Sta. St. Joe N.F. National Forest INT. County Shoshone. 5 miles E. Freezout Saddle.": (47.0885, -115.5028),
    "Makison Flats": (50.3000, -117.8000),
    "Makison Flats, Arrow Park": (50.3000, -117.8000),
    "Deep woods, Selkirk Mts.": (49.0000, -117.0000),
    "Quesnel Dam": (52.9814, -122.4891),
    "Kitsumgallum River, north of Luncheon Creek": (54.5073, -128.5581),
    "Wells area, Stout's Gulch": (53.1200, -121.5700),
    "Thompson-Nicola Regional District, Tranquille Range, near Kamloops": (50.7230, -120.5480),
    "Coast area, Garibaldi, Black Tusk Slopes": (49.9611, -123.0033),
    "East side of Warm River Canyon, near Henry's Fork of Snake River.": (44.0713, -111.2935),
    "Stevens Pass Region, Cascade Mountains.": (47.7458, -121.0896),
    "Nez Perce Natl. For.": (46.1841, -115.5322),
    "Alaska Highway, mile 355": (60.0954, -128.7983),
    "Wells area, Lowhee Ditch": (53.1000, -121.5833),
    "Burges Pass": (51.4220, -116.4850),
    "4 mi. E of Stevens Pass on U.S. 2 by turnoff in Spruce-Hemlock forest & alpine.": (47.7440, -120.9840),
    "Arrow Park, Makison Flats": (50.3000, -117.8000),
    "Columbia-Shuswap Regional District, 33 miles north of Revelstoke, by Mars Creek": (51.3167, -118.2167),
    "blue Mts.": (45.7167, -118.1000),
    "Wells area, Mount Wylie road": (53.1167, -121.5667),
    "Monashee Road above Inooklin Creek, just southeast of Banting Creek": (50.2375, -118.2428),
    "vicinity of Lake Waha": (46.2664, -116.9520),
    "Blue Mts..": (45.7167, -118.1000),  # likely a formatting variant
    "below Crater Lake.": (42.9000, -122.1333),
}

In [25]:
def fill_manual_coords(row):
    locality = row['locality']
    if pd.isna(row['decimalLatitude']) and pd.isna(row['decimalLongitude']) and locality in manual_geocodes:
        lat, lon = manual_geocodes[locality]
        row['decimalLatitude'] = lat
        row['decimalLongitude'] = lon
    return row

test = test.apply(fill_manual_coords, axis=1)

test[test['decimalLatitude'].isna() & test['decimalLongitude'].isna()]['municipality'].value_counts()

municipality
Green Mountain, near Haylmore.                                                                                        2
Blue Mts.                                                                                                             1
Vicinity of Helena.                                                                                                   1
Rogue river valley.                                                                                                   1
No specific locality recorded.                                                                                        1
Above the government camp, Mt. hood.                                                                                  1
Owyhee Mountains                                                                                                      1
Mt. hood, wapinilia pass.                                                                                             1
Slide overlooking Meager Cr

In [26]:
manual_geocodes_municipality = {
    "Green Mountain, near Haylmore.": (50.6983, -122.4821),
    "Blue Mts.": (45.7167, -118.1000),
    "Vicinity of Helena.": (46.5958, -112.0270),
    "Rogue river valley.": (42.4331, -123.3286),
    "Above the government camp, Mt. hood.": (45.3106, -121.7526),
    "Owyhee Mountains": (42.6197, -116.7894),
    "Mt. hood, wapinilia pass.": (45.1600, -121.7810),
    "Slide overlooking Meager Creek, ca. 1 mile above its junction with Lillooet River.": (50.4675, -123.4753),
    "Mile 355, Alaska highway.": (60.0954, -128.7983),
    "Mile 195, Alaska highway.": (59.3769, -126.9608),
    "10 miles north of Summit Lake on Hart Highway.": (55.3033, -122.6625),
    "Arrow park, fisher creek.": (50.3210, -117.8820),
    "One mile N of parsnip river along the hart highway.": (55.6833, -122.5833),
    "Takla Lake about 9 miles northwest of Takla Landing.": (55.7660, -126.2500),
    "Olympic Mountains. Elwha Valley.": (47.9714, -123.6061),
    "Along trail to forest fire lookout station, mcbride.": (53.3072, -120.1750),
    "Arrow park, stevens creek.": (50.3070, -117.8700),
    "Arrow park, makison flats.": (50.3000, -117.8000),
    "Dam Mt.": (50.0719, -123.0833),
    "Stevens Pass Region, Cascade Mountains.": (47.7458, -121.0896),
    "Echo Lake, Rocky Mountains, Flathead National Forest.": (47.5441, -113.9427),
    "Cameron lake, Waterton Lakes National Park.": (49.0239, -113.8982),
    "Inter Fork of White River, Mount Rainier.": (46.8535, -121.7017),
    "Near hope.": (49.3824, -121.4409),
    "Cascade Mountains, Mount Rainier and adjacent mountains.": (46.8523, -121.7603),
    "below Crater Lake.": (42.9000, -122.1333),
    "Burman Lake, drainage, Burman River. Vancouver Island.": (49.6887, -126.0917),
    "Trail that runs along east shore of burges pass, emerald lake.": (51.4206, -116.4728),
    "1.7 miles south of Lakeview.": (41.9900, -120.5581),
    "Aeta Lake.": (49.7683, -125.0020),
    "Burges pass.": (51.4220, -116.4850),
    "Trail to Elk Summit to Kooskooska. Selway Forest. (county unknown)": (46.1476, -114.7625),
    "Glacier National Park, Sun Camp.": (48.6101, -113.4541),
    "Between horsefly & crooked lake.": (52.2667, -121.1167),
    "Missoula County, north of upper Holland Lake, swan range, ca. 80 miles NE of Missoula, township 30 n, r15w, s. 29.": (47.4167, -113.4167),
    "Mt. richards, NE of 60 degree slope, Waterton Lakes National Park.": (49.1167, -113.7833),
    "Azouzetta Lake on Hart Highway.": (55.3667, -122.7167),
    "Seven Devils Mountains Nez Perce National Forest.": (45.2506, -116.6706),
    "Ant Basin, Idaho, East Lago.": (42.1722, -111.3833),
    "Near South shore of Priest Lake.": (48.5272, -116.9150),
    "Sumup Cabin, southwest of Mount Baldy, 9 miles northwest of Ross Spur.": (48.3039, -117.2667),
    "Monashee road above inooklin creek, just SE of banting creek.": (50.2375, -118.2428),
    "Near kootenay florence mine, ainsworth.": (49.7589, -116.9256),
    "West side of Mountain Revelstoke.": (51.0312, -118.1790),
    "Along Hope-Princeton Highway where road crosses Skaist River.": (49.3000, -120.7833),
    "Circa. 4.25 miles northeast by east of Falkland.": (50.5383, -119.1667),
    "East of takatoot l.": (54.3167, -126.2500),
    "Shore of Blackwater Lake, circa 32 miles north of Golden.": (51.7833, -116.9667),
    "Outpost lake trail, Jasper National Park.": (52.8442, -117.9394),
    "On Bolean Lake Trail, circa. 4 miles northeast by east of Falkland.": (50.5400, -119.1500),
    "Glacier National Park, south slope of Quartz Ridge, north of Quartz Lake.": (48.8667, -114.0000),
    "Headwaters of Jocko River": (47.2000, -114.0000),
    "Near Lake Bootahnie, Marble Mountains.": (41.6528, -123.7283),
    "Near south shore of Priest Lake.": (48.5272, -116.9150)
}

In [27]:
def fill_from_municipality(row):
    if pd.isna(row['decimalLatitude']) and pd.isna(row['decimalLongitude']):
        muni = row['municipality']
        if pd.notna(muni) and muni in manual_geocodes_municipality:
            lat, lon = manual_geocodes_municipality[muni]
            row['decimalLatitude'] = lat
            row['decimalLongitude'] = lon
    return row

test = test.apply(fill_from_municipality, axis=1)
test[test['decimalLatitude'].isna() & test['decimalLongitude'].isna()]['municipality'].value_counts()

municipality
No specific locality recorded.       1
[no locality given on the label].    1
Name: count, dtype: int64

In [17]:
test[test['decimalLatitude'].isna() & test['decimalLongitude'].isna()].shape

(21, 32)

In [31]:
test[test['decimalLatitude'].isna() & test['decimalLongitude'].isna()]

,reproductiveCondition,occurrenceStatus,eventDate,eventTime,startDayOfYear,endDayOfYear,year,month,day,verbatimEventDate,habitat,higherGeography,continent,countryCode,stateProvince,county,municipality,locality,verbatimLocality,verbatimElevation,minimumDistanceAboveSurfaceInMeters,maximumDistanceAboveSurfaceInMeters,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,georeferencedDate,dateIdentified,elevation,elevationAccuracy,hasCoordinate,hasGeospatialIssues,used_llm
24,NaN,PRESENT,1800-01-01/2008-04-25,1800-01-01/2008-04-25,1.0,116.0,NaN,NaN,NaN,not recorded,NaN,no higher geography recorded,NaN,NaN,NaN,NaN,NaN,No specific locality recorded.,No verbatim locality recorded,NaN,NaN,NaN,NaN,NaN,NaN,2012-07-08 00:00:00,2008-04-25T00:00:00,NaN,NaN,False,False,False
1959,NaN,PRESENT,1892-06-20,NaN,172.0,172.0,1892.0,6.0,20.0,NaN,NaN,NaN,NaN,US,Washington,NaN,NaN,[no locality given on the label].,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1944-01-01T00:00:00,NaN,NaN,False,False,False
5179,NaN,PRESENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,California,NaN,NaN,[no additional data],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
5270,NaN,PRESENT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASIA,AF,NaN,NaN,No specific locality recorded.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
5480,NaN,PRESENT,1892-06-20,NaN,172.0,172.0,1892.0,6.0,20.0,NaN,NaN,NaN,NaN,US,Washington,NaN,[no locality given on the label].,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False
6452,NaN,PRESENT,1892-06-20,NaN,172.0,172.0,1892.0,6.0,20.0,NaN,NaN,NaN,NaN,US,Washington,NaN,NaN,[no locality given on the label].,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False


In [33]:
test = test.dropna(subset=['decimalLatitude', 'decimalLongitude'])  # Final clean-up
test[test['decimalLatitude'].isna() & test['decimalLongitude'].isna()].shape

(0, 32)

In [36]:
test.columns

Index(['reproductiveCondition', 'occurrenceStatus', 'eventDate', 'eventTime',
       'startDayOfYear', 'endDayOfYear', 'year', 'month', 'day',
       'verbatimEventDate', 'habitat', 'higherGeography', 'continent',
       'countryCode', 'stateProvince', 'county', 'municipality', 'locality',
       'verbatimLocality', 'verbatimElevation',
       'minimumDistanceAboveSurfaceInMeters',
       'maximumDistanceAboveSurfaceInMeters', 'decimalLatitude',
       'decimalLongitude', 'coordinateUncertaintyInMeters',
       'georeferencedDate', 'dateIdentified', 'elevation', 'elevationAccuracy',
       'hasCoordinate', 'hasGeospatialIssues', 'used_llm'],
      dtype='object')

In [38]:
test['higherGeography'].value_counts()

higherGeography
U.S.A., Montana, Missoula County                          43
U.S.A., Montana, Ravalli County                           24
U.S.A., Montana, Mineral County                           22
U.S.A., Montana, Flathead County                          21
U.S.A., Idaho, Valley County                              19
                                                          ..
North America, United States, Montana                      1
North America, Canada, British Columbia                    1
North America, United States, Oregon, Josephine            1
North America;Canada;British Columbia;Vancouver Island     1
North America; USA; Montana                                1
Name: count, Length: 80, dtype: int64

In [ ]:
# Drop columns we no longer need since we have clean lat/long coordinates
columns_to_drop = ['higherGeography', 'continent', 'countryCode', 'stateProvince', 
                  'county', 'municipality', 'locality', 'verbatimLocality',
                  'verbatimElevation', 'minimumDistanceAboveSurfaceInMeters',
                  'maximumDistanceAboveSurfaceInMeters',
                  'used_llm']

test = test.drop(columns=columns_to_drop)


In [23]:
final = test.copy()
final.head(3)

display(int(final['reproductiveCondition'].value_counts().values.sum()) / final.shape[0] * 100)
display(int(final['elevation'].value_counts().values.sum()) / final.shape[0] * 100)

8.886036228783341

6.4898017401226635

In [31]:
test.to_csv('../data/processed/coordinates.csv', index=False)